In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from ultralytics import YOLO
import time
import threading

# Liste des objets à détecter
OBJETS_SURVEILLES = {
    'Sunglasses': 505,  # Lunettes de soleil
    'Hat': 243,        # Chapeau général
    'Shorts': 456,     # Short
    'Skirt': 463,      # Jupe
    'Miniskirt': 334,  # Mini-jupe
    'Helmet': 248,     # Casque
    'Kitchen Knife': 292,  # Couteau de cuisine
    'Shotgun': 457,    # Arme à feu
    'Handgun': 238     # Pistolet
}

def initialiser_camera():
    """Initialise la webcam"""
    return cv2.VideoCapture(0)

def charger_modele():
    """Charge le modèle YOLO"""
    return YOLO("yolov8n-oiv7.pt")

def detecter_objets(frame, model):
    """Détecte les objets dans la frame"""
    results = model(frame)
    detections = []

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Récupérer l'ID de classe
            cls_id = int(box.cls[0])

            # Vérifier si l'objet est dans notre liste de surveillance
            for nom_objet, id_classe in OBJETS_SURVEILLES.items():
                if cls_id == id_classe:
                    # Récupérer les coordonnées et la confiance
                    x1, y1, x2, y2 = box.xyxy[0]
                    confidence = float(box.conf[0])

                    if confidence > 0.4:  # Seuil de confiance
                        detections.append({
                            'objet': nom_objet,
                            'confiance': confidence,
                            'coords': (int(x1), int(y1), int(x2), int(y2))
                        })

    return detections

def dessiner_detections(frame, detections):
    """Dessine les boîtes de détection et les labels sur la frame"""
    for det in detections:
        x1, y1, x2, y2 = det['coords']
        nom = f"{det['objet']} {det['confiance']:.2f}"

        # Dessiner la boîte
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Ajouter le label
        cv2.putText(frame, nom, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 0, 255), 2)

    return frame

def traiter_camera(model):
    """Traite les images en temps réel de la caméra"""
    cap = initialiser_camera()
    if not cap.isOpened():
        print("Erreur: Impossible d'ouvrir la caméra")
        return

    print("Détection en cours... Appuyez sur 'q' pour quitter.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Erreur: Impossible de lire la frame")
            break

        # Détection des objets
        detections = detecter_objets(frame, model)

        # Dessin des détections
        frame = dessiner_detections(frame, detections)

        # Affichage du résultat
        cv2.imshow('Detection en temps reel', frame)

        # Quitter avec 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Nettoyage
    cap.release()
    cv2.destroyAllWindows()

def importer_image():
    """Permet d'importer une image et de lancer la détection"""
    chemin_image = filedialog.askopenfilename(title="Choisir une image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
    if chemin_image:
        image = cv2.imread(chemin_image)
        model = charger_modele()
        detections = detecter_objets(image, model)
        image = dessiner_detections(image, detections)
        cv2.imshow("Image détectée", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def demarrer_camera():
    """Démarre la caméra dans un thread"""
    model = charger_modele()
    thread_camera = threading.Thread(target=traiter_camera, args=(model,))
    thread_camera.start()

def create_interface():
    """Crée l'interface graphique avec tkinter"""
    root = tk.Tk()
    root.title("Détection d'objets")

    # Bouton pour importer une image
    bouton_importer_image = tk.Button(root, text="Importer une image", command=importer_image)
    bouton_importer_image.pack(pady=10)

    # Bouton pour démarrer la caméra
    bouton_camera = tk.Button(root, text="Démarrer la caméra", command=demarrer_camera)
    bouton_camera.pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    create_interface()



0: 640x480 2 Footwears, 1 Woman, 78.7ms
Speed: 0.0ms preprocess, 78.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x448 2 Footwears, 1 Shorts, 1 Woman, 78.2ms
Speed: 0.0ms preprocess, 78.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x640 1 Dress, 1 Skirt, 1 Woman, 109.8ms
Speed: 15.6ms preprocess, 109.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
